In [392]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Standard
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import signal
from PIL import Image
import scipy

# # Tensorflow and Keras
# from keras.datasets import mnist
# from keras.models import Sequential, Model, Input
# from keras.layers import Dense, Dropout, Activation
# from keras.layers import Conv2D, MaxPooling2D, Flatten
# from tensorflow.keras.optimizers import SGD
# from keras.regularizers import l2
# import tensorflow as tf

# Xception because other model not working
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications import Xception
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, RMSprop

from keras_preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

#os.chdir('../')
from src import image_preprocess as ip
from src import models

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Start with histology data
for each 40X, 100X, 200X, 400X, have 644 items for benign and 1300 for malignant

Augment our benign data since we only have ~650 images. This might not be necessary.

In [50]:
files = [f for f in os.listdir('data/Histology/100X/benign')]

for f in files:
    img = Image.open(os.path.join('data/Histology/100X/benign',f))
    x = ip.reshape_image(img)
    ip.create_new_images(x)    


In [40]:
root_dir = '/home/maureen/Documents/Galvanize/Capstone1/Capstone3/Cancer_Prediction'

## Make the model using simplecnn.py

In [149]:
# Assuming input shape = (width, height, channels) NOT (rows, cols, channels)
model = simple_cnn.create_model(input_size=(500,328,3), n_categories=2)

In [185]:
training_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                      horizontal_flip=True) 
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [319]:
train_path = 'data/Histology/100X/train'
val_path = 'data/Histology/100X/validation'
test_path = 'data/Histology/100X/test'

In [320]:
train_generator = training_datagen.flow_from_directory(train_path, 
                                                       target_size=(500, 328), 
                                                       batch_size=16)
validation_generator = validation_datagen.flow_from_directory(val_path, 
                                                              target_size=(500,328), 
                                                              batch_size=16)
test_generator = test_datagen.flow_from_directory(test_path,
                                                 target_size=(500,328),
                                                 batch_size=16)

Found 1642 images belonging to 2 classes.
Found 427 images belonging to 2 classes.
Found 24 images belonging to 1 classes.


In [153]:
model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

In [158]:
from tensorflow.keras.callbacks import TensorBoard
%load_ext tensorboard

callback = keras.callbacks.ModelCheckpoint(filepath='models/simplecnn_whc.h5', save_best_only=True)
tensorboard_callback = keras.callbacks.TensorBoard(log_dir="logs")

In [159]:
model.fit_generator(train_generator,
                   steps_per_epoch=1666//16,
                   epochs=3,
                   validation_data=validation_generator,
                   validation_steps=415//16,
                   callbacks=[callback, tensorboard_callback])

Epoch 1/3
104/104 [==============================] - 429s 4s/step - loss: 4.7324 - accuracy: 0.6897 - val_loss: 4.7928 - val_accuracy: 0.6875
Epoch 2/3
104/104 [==============================] - 421s 4s/step - loss: 4.7503 - accuracy: 0.6927 - val_loss: 2.8757 - val_accuracy: 0.6892
Epoch 3/3
104/104 [==============================] - 420s 4s/step - loss: 4.7965 - accuracy: 0.6897 - val_loss: 6.7100 - val_accuracy: 0.6892


## Try Transfer Learning!

In [391]:
# Run on Histology data
mag = ['40X', '100X', '200X', '400X']
predictions = []
filenames = []

for m in mag:
    train_path = f'data/Histology/{m}/train'
    val_path = f'data/Histology/{m}/validation'
    test_path = f'data/Histology/{m}/test'
    
    model = models.Xception_model(500,368)
    model.compile_model(train_path, val_path)
    model.fit()
    files, predict = model.predict(test_path)
    
    filenames.append(files)
    predictions.append(predict)

Found 1577 images belonging to 2 classes.
Found 399 images belonging to 2 classes.
Epoch 1/4


ValueError: in user code:

    /home/maureen/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /home/maureen/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/maureen/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/maureen/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/maureen/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:543 train_step  **
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    /home/maureen/anaconda3/lib/python3.7/site-packages/tensorf

In [378]:
predictions = np.asarray(predictions)
predictions.shape
np.save('filenames.txt', filenames)
# with open('predictionsrmsprop.txt', 'w') as f:
#     f.write('\n'.join(predictions))

In [377]:
filenames

[['test/SOB_B_F-14-25197-40-041.png',
  'test/SOB_B_F-14-29960AB-40-014.png',
  'test/SOB_B_PT-14-21998AB-40-056.png',
  'test/SOB_B_PT-14-22704-40-036.png',
  'test/SOB_B_PT-14-29315EF-40-009.png',
  'test/SOB_B_TA-14-13200-40-006.png',
  'test/SOB_B_TA-14-3411F-40-015.png',
  'test/SOB_M_DC-14-2523-40-012.png',
  'test/SOB_M_DC-14-2773-40-004.png',
  'test/SOB_M_DC-14-2980-40-004.png',
  'test/SOB_M_DC-14-2985-40-001.png',
  'test/SOB_M_DC-14-3909-40-001.png',
  'test/SOB_M_DC-14-4372-40-005.png',
  'test/SOB_M_DC-14-5287-40-008.png',
  'test/SOB_M_DC-14-5694-40-005.png',
  'test/SOB_M_DC-14-5695-40-011.png',
  'test/SOB_M_DC-14-6241-40-014.png',
  'test/SOB_M_DC-14-8168-40-004.png',
  'test/SOB_M_DC-14-9461-40-012.png'],
 ['test/SOB_B_F-14-23222AB-100-016.png',
  'test/SOB_B_F-14-25197-100-030.png',
  'test/SOB_B_F-14-29960AB-100-013.png',
  'test/SOB_B_PT-14-21998AB-100-062.png',
  'test/SOB_B_PT-14-22704-100-036.png',
  'test/SOB_B_PT-14-29315EF-100-013.png',
  'test/SOB_B_TA-14-3

In [355]:
# Using model class!! YAY!!
train_path = 'data/Histology/100X/train'
val_path = 'data/Histology/100X/validation'
test_path = 'data/Histology/100X/test'

model = models.Xception_model(500,368)
print(model.channels, model.classes)

# Compile Model
model.compile_model(train_path, val_path)
model.train_generator.samples

In [359]:
model.fit()

81/98 [=======================>......] - ETA: 1:13 - loss: 0.5854 - accuracy: 0.8037

KeyboardInterrupt: 

In [351]:
model.predict(test_path)

Found 24 images belonging to 1 classes.


(['test/SOB_B_F-14-23222AB-100-016.png',
  'test/SOB_B_F-14-25197-100-030.png',
  'test/SOB_B_F-14-29960AB-100-013.png',
  'test/SOB_B_PT-14-21998AB-100-062.png',
  'test/SOB_B_PT-14-22704-100-036.png',
  'test/SOB_B_PT-14-29315EF-100-013.png',
  'test/SOB_B_TA-14-3411F-100-015.png',
  'test/SOB_M_DC-14-10926-100-004.png',
  'test/SOB_M_DC-14-11031-100-008.png',
  'test/SOB_M_DC-14-11520-100-001.png',
  'test/SOB_M_DC-14-11951-100-003.png',
  'test/SOB_M_DC-14-12312-100-005.png',
  'test/SOB_M_DC-14-2523-100-026.png',
  'test/SOB_M_DC-14-2773-100-014.png',
  'test/SOB_M_DC-14-2980-100-006.png',
  'test/SOB_M_DC-14-2985-100-007.png',
  'test/SOB_M_DC-14-3909-100-009.png',
  'test/SOB_M_DC-14-4372-100-004.png',
  'test/SOB_M_DC-14-5287-100-003.png',
  'test/SOB_M_DC-14-5694-100-006.png',
  'test/SOB_M_DC-14-5695-100-019.png',
  'test/SOB_M_DC-14-6241-100-004.png',
  'test/SOB_M_DC-14-8168-100-003.png',
  'test/SOB_M_DC-14-9461-100-003.png'],
 array([[2.831617e-03, 9.971684e-01],
        

## All the code for Xception Class

In [321]:
# Try new model because previous isn't working
# Using softmax so that we'll get probabilities for predictions
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout
from tensorflow.keras.models import Model

def create_transfer_model(input_size, n_categories, weights = 'imagenet'):
        # note that the "top" is not included in the weights below
        base_model = Xception(weights=weights,
                          include_top=False,
                          input_shape=input_size)
        
        model = base_model.output
        model = GlobalAveragePooling2D()(model)
        predictions = Dense(n_categories, activation='softmax')(model)
        model = Model(inputs=base_model.input, outputs=predictions)
        
        return model

In [322]:
model = create_transfer_model((500,328,3),2) 

In [323]:
def print_model_properties(model, indices = 0):
    for i, layer in enumerate(model.layers[indices:]):
        print(i+indices, layer.name,layer.trainable)

In [83]:
print_model_properties(model)

0 input_5 True
1 block1_conv1 True
2 block1_conv1_bn True
3 block1_conv1_act True
4 block1_conv2 True
5 block1_conv2_bn True
6 block1_conv2_act True
7 block2_sepconv1 True
8 block2_sepconv1_bn True
9 block2_sepconv2_act True
10 block2_sepconv2 True
11 block2_sepconv2_bn True
12 conv2d_16 True
13 block2_pool True
14 batch_normalization_16 True
15 add_48 True
16 block3_sepconv1_act True
17 block3_sepconv1 True
18 block3_sepconv1_bn True
19 block3_sepconv2_act True
20 block3_sepconv2 True
21 block3_sepconv2_bn True
22 conv2d_17 True
23 block3_pool True
24 batch_normalization_17 True
25 add_49 True
26 block4_sepconv1_act True
27 block4_sepconv1 True
28 block4_sepconv1_bn True
29 block4_sepconv2_act True
30 block4_sepconv2 True
31 block4_sepconv2_bn True
32 conv2d_18 True
33 block4_pool True
34 batch_normalization_18 True
35 add_50 True
36 block5_sepconv1_act True
37 block5_sepconv1 True
38 block5_sepconv1_bn True
39 block5_sepconv2_act True
40 block5_sepconv2 True
41 block5_sepconv2_bn Tru

Change head

In [324]:
def change_trainable_layers(model, trainable_index):
    for layer in model.layers[:trainable_index]:
        layer.trainable = False
    for layer in model.layers[trainable_index:]:
        layer.trainable = True

In [325]:
_ = change_trainable_layers(model, 132)
print_model_properties(model, 130)

130 block14_sepconv2_bn False
131 block14_sepconv2_act False
132 global_average_pooling2d_16 True
133 dense_13 True


In [326]:
# Compile model. Changed lr from 0.0005 to 0.005
model.compile(optimizer=RMSprop(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [327]:
# Train model can we use fit generator?
model.fit_generator(train_generator,
                   steps_per_epoch=1666//16,
                   epochs=1,
                   validation_data=validation_generator,
                   validation_steps=415//16)
model.save_weights('models/weights.h5')
model.save('models/transfermodel.h5')

  6/104 [>.............................] - ETA: 3:59 - loss: 1.6539 - accuracy: 0.6458

KeyboardInterrupt: 

In [ ]:
## Make predictions!

In [205]:
path = 'data/Histology/100X/test/test/SOB_B_TA-14-13200-100-001.png'

In [206]:
# Single image
img = Image.open(path)
x = img_to_array(img)
images = np.array([x])
images.shape

(1, 328, 500, 3)

In [212]:
model.predict(images)

array([[0.040003, 0.959997]], dtype=float32)

In [216]:
# predict on entire folder
predictions = model.predict(test_generator)
predictions

array([[0.972714, 0.027286],
       [0.032123, 0.967877],
       [0.074094, 0.925906],
       [0.947288, 0.052712],
       [0.375703, 0.624297],
       [0.122902, 0.877098],
       [0.025837, 0.974163],
       [0.96413 , 0.03587 ],
       [0.970251, 0.029749],
       [0.016283, 0.983717],
       [0.034223, 0.965777],
       [0.053022, 0.946978],
       [0.044166, 0.955834],
       [0.038925, 0.961075],
       [0.014544, 0.985456],
       [0.004985, 0.995015],
       [0.049999, 0.950001]], dtype=float32)

In [204]:
test_generator.filenames

['test/SOB_B_TA-14-13200-100-001.png',
 'test/SOB_B_TA-14-13200-100-002.png',
 'test/SOB_B_TA-14-13200-100-003.png',
 'test/SOB_B_TA-14-13200-100-004.png',
 'test/SOB_B_TA-14-13200-100-005.png',
 'test/SOB_M_MC-14-13413-100-030.png',
 'test/SOB_M_MC-14-13413-100-031.png',
 'test/SOB_M_MC-14-13413-100-032.png',
 'test/SOB_M_MC-14-13413-100-033.png',
 'test/SOB_M_MC-14-13413-100-034.png',
 'test/SOB_M_MC-14-13413-100-035.png',
 'test/SOB_M_MC-14-13413-100-036.png',
 'test/SOB_M_MC-14-13413-100-037.png',
 'test/SOB_M_MC-14-13413-100-038.png',
 'test/SOB_M_MC-14-13413-100-039.png',
 'test/SOB_M_MC-14-13413-100-040.png',
 'test/SOB_M_MC-14-13413-100-041.png']